In [ ]:
#!pip install gensim==3.6.0

In [1]:
import gensim
gensim.__version__

'3.6.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir("/content/drive/MyDrive/Datasets")

In [4]:
import numpy as np
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import  Word2Vec
from sklearn.metrics import roc_auc_score

In [5]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t')

In [6]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
################### Replacing Apostrophe to Word #################
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"don\'t", "do not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
###################################################################

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
stops = stopwords.words('english')

In [10]:
corpus = []
for i in range(0, dataset.shape[0]):
    review = dataset['Review'][i]
    review = decontracted(review)    
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
#    ps = PorterStemmer()
    review = review.split()
#    review = [word for word in review if not word in set(stops)]
#    review = ' '.join(review)
#    review = [review]
    if review == []:
      review = ['something']
    corpus.append(review)

In [11]:
len(corpus)

1000

In [12]:
corpus[:3]

[['wow', 'loved', 'this', 'place'],
 ['crust', 'is', 'not', 'good'],
 ['not', 'tasty', 'and', 'the', 'texture', 'was', 'just', 'nasty']]

CBOW: sg = 0
Skip-Gram: sg = 1

In [13]:
model_r =  Word2Vec(corpus, min_count=1,sg=0)

In [ ]:
means = []
for i in corpus :
    row_means = np.mean(model_r[i],axis=0)
    means.append(row_means)
means = np.array(means) 

X = means
y = dataset.iloc[:, 1]

In [ ]:
X.shape

In [ ]:
y.shape

In [15]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, 
                                                    random_state=2022,
                                                    stratify=y)

from xgboost import XGBClassifier
model_rf = XGBClassifier(random_state=2022)
model_rf.fit( X_train , y_train )

XGBClassifier(random_state=2022)

**Model Evaluation**


In [16]:
y_pred_prob = model_rf.predict_proba(X_test)[:,1]

print(roc_auc_score(y_test, y_pred_prob))

0.5515555555555555
